# Baseline NL→SQL Evaluation (Zero-shot vs Few-shot)

This notebook is the baseline replication block for Ojuri et al. (2025): it measures prompting effects (`k=0` vs `k=3`) on the same 200-item ClassicModels test set using an open-source local stack.

Primary role in dissertation:
- establish non-fine-tuned reference metrics,
- isolate prompting gains before QLoRA,
- provide controlled inputs for later paired comparisons.

This notebook runs the VA/EX baseline over `data/classicmodels_test_200.json` and saves outputs under `results/baseline/`.


Run this setup in a fresh GPU runtime, then restart before continuing.

In [ ]:
%%bash
set -e
export PIP_DEFAULT_TIMEOUT=120

# Clean conflicting preinstalls
pip uninstall -y torch torchvision torchaudio bitsandbytes triton transformers accelerate peft trl datasets numpy pandas fsspec requests google-auth scipy scikit-learn || true

# Base deps
pip install -q --no-cache-dir --force-reinstall \
  numpy==1.26.4 pandas==2.2.1 scipy scikit-learn \
  fsspec==2024.5.0 requests==2.31.0 google-auth==2.43.0

# Torch + CUDA 12.1
pip install -q --no-cache-dir --force-reinstall \
  torch==2.3.1+cu121 torchvision==0.18.1+cu121 torchaudio==2.3.1+cu121 \
  --index-url https://download.pytorch.org/whl/cu121

# bitsandbytes + triton + HF stack
pip install -q --no-cache-dir --force-reinstall \
  bitsandbytes==0.43.3 triton==2.3.1 \
  transformers==4.44.2 accelerate==0.33.0 peft==0.17.0 trl==0.9.6 datasets==2.20.0

echo "Setup complete. Restart runtime once, then run the rest of the notebook top-to-bottom."


After restart, continue with DB/auth, schema, model, and eval cells.

Prompt/eval: build prompts (system+schema+k exemplars), generate SQL, postprocess, and compute VA/EX/EM.

**Docs (schema prompts):** NL→SQL schema-grounded prompting survey https://arxiv.org/abs/2410.06011; Spider-style listings.

In [ ]:
import os, sys, shutil
from pathlib import Path

# If this notebook is opened directly in Colab (not from a cloned repo), clone the repo first.
if Path("data/classicmodels_test_200.json").exists() is False and Path("/content").exists():
    repo_dir = Path("/content/NLtoSQL")
    if repo_dir.exists():
        shutil.rmtree(repo_dir)
    !git clone https://github.com/MacKenzieOBrian/NLtoSQL.git "{repo_dir}"
    os.chdir(repo_dir)

# Ensure repo root is on sys.path for `import nl2sql`
sys.path.insert(0, os.getcwd())
print("cwd:", os.getcwd())


## Install dependencies (Colab)

This repo pins versions in `requirements.txt` to reduce Colab binary drift.
After installation, restart the runtime (Runtime → Restart runtime), then run this notebook again from the top.


In [ ]:
try:
    import google.colab  # noqa: F401
    IN_COLAB = True
except Exception:
    IN_COLAB = False

if IN_COLAB:
    !pip -q install -r requirements.txt
else:
    print("Not in Colab; ensure requirements are installed.")


In [ ]:
# Colab-only: authenticate with GCP (safe to skip locally)
try:
    from google.colab import auth
except ModuleNotFoundError:
    auth = None

if auth:
    auth.authenticate_user()
else:
    print("Not running in Colab; ensure ADC or service account auth is configured.")


In [ ]:
# Hugging Face auth (gated model)
import os

hf_token = os.getenv("HF_TOKEN") or os.getenv("HUGGINGFACE_HUB_TOKEN")
if hf_token:
    os.environ["HUGGINGFACE_HUB_TOKEN"] = hf_token
    print("Using HF token from env")
else:
    try:
        from huggingface_hub import notebook_login
        notebook_login()
    except Exception as e:
        print("HF auth not configured:", e)


**Docs (auth/DB):** Cloud SQL connector pattern https://cloud.google.com/sql/docs/mysql/connect-run; SQLAlchemy creator hook https://docs.sqlalchemy.org/en/20/core/engines.html#custom-dbapi-connect.

In [ ]:
import json
from getpass import getpass

INSTANCE_CONNECTION_NAME = os.getenv("INSTANCE_CONNECTION_NAME")
DB_USER = os.getenv("DB_USER")
DB_PASS = os.getenv("DB_PASS")
DB_NAME = os.getenv("DB_NAME", "classicmodels")

if not INSTANCE_CONNECTION_NAME:
    INSTANCE_CONNECTION_NAME = input("Enter INSTANCE_CONNECTION_NAME: ").strip()
if not DB_USER:
    DB_USER = input("Enter DB_USER: ").strip()
if not DB_PASS:
    DB_PASS = getpass("Enter DB_PASS: ")

print("Using DB:", DB_NAME)

test_set = json.loads(open("data/classicmodels_test_200.json", "r", encoding="utf-8").read())
print("Loaded test items:", len(test_set))


**Docs (auth/DB):** Cloud SQL connector pattern https://cloud.google.com/sql/docs/mysql/connect-run; SQLAlchemy creator hook https://docs.sqlalchemy.org/en/20/core/engines.html#custom-dbapi-connect.

In [ ]:
from nl2sql.db import create_engine_with_connector

engine, connector = create_engine_with_connector(
    instance_connection_name=INSTANCE_CONNECTION_NAME,
    user=DB_USER,
    password=DB_PASS,
    db_name=DB_NAME,
)

print("Engine ready")


**Ref:** HF Transformers 4-bit load with BitsAndBytes (quantization docs: https://huggingface.co/docs/transformers/main_classes/quantization). Mirrors PEFT/QLoRA examples for gated Llama 3; keeps model within Colab GPU VRAM.

**Docs (model load):** HF 4-bit NF4 quantization https://huggingface.co/docs/transformers/main_classes/quantization; PEFT/QLoRA https://huggingface.co/docs/peft/.

In [ ]:
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig

MODEL_ID = "meta-llama/Meta-Llama-3-8B-Instruct"

print("Loading tokenizer...")
tok = AutoTokenizer.from_pretrained(MODEL_ID, token=True)
if tok.pad_token is None:
    tok.pad_token = tok.eos_token

# Try 4-bit loading (fallback to fp/bf16)
try:
    bnb_config = BitsAndBytesConfig(
        load_in_4bit=True,
        bnb_4bit_use_double_quant=True,
        bnb_4bit_quant_type="nf4",
        bnb_4bit_compute_dtype=torch.bfloat16,
    )
    print("Attempting 4-bit quantized load...")
    model = AutoModelForCausalLM.from_pretrained(
        MODEL_ID,
        quantization_config=bnb_config,
        torch_dtype=torch.bfloat16,
        device_map="auto",
        token=True,
    )
except Exception as e:
    print("4-bit load failed, falling back. Error:")
    print(e)
    model = AutoModelForCausalLM.from_pretrained(
        MODEL_ID,
        torch_dtype=torch.bfloat16 if torch.cuda.is_available() else torch.float32,
        device_map="auto" if torch.cuda.is_available() else None,
        token=True,
    )

model.generation_config.do_sample = False
model.generation_config.num_beams = 1

print("Model device:", model.device)


**Ref:** Schema summary helper from this repo (`nl2sql.schema`) following schema-grounded NL→SQL prompting practice (see survey: https://arxiv.org/abs/2410.06011). Including table/column context reduces column/join errors in zero/few-shot prompts.

**Docs (schema prompts):** NL→SQL schema-grounded prompting survey https://arxiv.org/abs/2410.06011; Spider-style listings.

In [ ]:
from nl2sql.schema import build_schema_summary

SCHEMA_SUMMARY = build_schema_summary(engine, db_name=DB_NAME, max_cols_per_table=50)
print("Schema summary built (chars):", len(SCHEMA_SUMMARY))


**Ref:** Repo evaluation harness (`nl2sql.eval`) implementing VA/EX/EM execution-based metrics (semantic accuracy per EMNLP'20 TS work: https://aclanthology.org/2020.emnlp-main.29/).

**Docs (prompt/eval):** ICL patterns https://arxiv.org/abs/2005.14165; execution-based metrics (VA/EX) https://aclanthology.org/2020.emnlp-main.29/.

In [ ]:
import subprocess
from pathlib import Path

from nl2sql.eval import eval_run

try:
    commit = subprocess.check_output(["git", "rev-parse", "--short", "HEAD"]).decode().strip()
except Exception:
    commit = "unknown"
run_metadata = {
    "commit": commit,
    "model_id": MODEL_ID,
    "notebook": "02_baseline_prompting_eval.ipynb",
}

Path("results/baseline").mkdir(parents=True, exist_ok=True)

# Quick smoke test
# _ = eval_run(test_set=test_set, k=0, limit=20, seed=7, engine=engine, model=model, tokenizer=tok, schema_summary=SCHEMA_SUMMARY,
#              save_path="results/baseline/results_zero_shot_20.json", run_metadata=run_metadata)

# Full run (n=200)
zero_200 = eval_run(
    test_set=test_set,
    exemplar_pool=test_set,
    k=0,
    limit=None,
    seed=7,
    engine=engine,
    model=model,
    tokenizer=tok,
    schema_summary=SCHEMA_SUMMARY,
    save_path="results/baseline/results_zero_shot_200.json",
    run_metadata=run_metadata,
    avoid_exemplar_leakage=True,
)

few_200 = eval_run(
    test_set=test_set,
    exemplar_pool=test_set,
    k=3,
    limit=None,
    seed=7,
    engine=engine,
    model=model,
    tokenizer=tok,
    schema_summary=SCHEMA_SUMMARY,
    save_path="results/baseline/results_few_shot_k3_200.json",
    run_metadata=run_metadata,
    avoid_exemplar_leakage=True,
)


In [ ]:
# Quick summary (reads the saved JSON outputs)
import json

zero = json.loads(open("results/baseline/results_zero_shot_200.json", "r", encoding="utf-8").read())
few  = json.loads(open("results/baseline/results_few_shot_k3_200.json", "r", encoding="utf-8").read())

print("Zero-shot:", "VA", round(zero["va_rate"], 3), "EM", round(zero.get("em_rate", 0.0), 3), "EX", round(zero["ex_rate"], 3))
print("Few-shot:",  "VA", round(few["va_rate"], 3),  "EM", round(few.get("em_rate", 0.0), 3),  "EX", round(few["ex_rate"], 3))
